In [86]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from math import floor, ceil
from pylab import rcParams
import re

%matplotlib inline
engine = sqlite3.connect('DB')

In [87]:
#Write code to convert CSV to Pandas. Pretty sure I lost it somewhere
test = pd.read_csv('./test.csv')
baseTrain = pd.read_csv('./train.csv')
featuresDf = pd.read_csv('./features.csv')

In [88]:
#Combining training data with the features
storesWithTrain = pd.read_sql("""
SELECT *
FROM trainDf
JOIN storesDf 
ON storesDf.Store = trainDf.Store
""", engine)
_, i = np.unique(storesWithTrain.columns, return_index = True)
storesWithTrain = storesWithTrain.iloc[:, i]
storesWithTrain.to_sql('storesWithTrain', engine, if_exists='replace', index = False)

everything = pd.read_sql("""
SELECT *
FROM storesWithTrain
JOIN featuresDf
ON storesWithTrain.Store = featuresDf.Store AND storesWithTrain.Date = featuresDf.Date
""", engine)

#Get all unique elements
_, i = np.unique(everything.columns, return_index = True)
everything = everything.iloc[:, i]

In [89]:
#Grabbing just the basic features that match with the testing data
baseTrain_y = baseTrain['Weekly_Sales']
baseTrain_x = baseTrain.drop('Weekly_Sales', axis = 1)
baseTrain_x['Date'] = baseTrain_x['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
baseTrain_x['Date'] = baseTrain_x['Date'].apply(lambda x: int(time.mktime(x.timetuple())) / 100000)

In [90]:
#Cleaning the data that has everything

#Parsing the data/convert data time to milliseconds
train_x = everything.drop_duplicates()
train_x = train_x.dropna().reset_index(drop = True)#Get rid of this line if you don't want to drop NaN

#Convert date string to date time object to get milliseconds
train_x['Date'] = train_x['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
train_x['Date'] = train_x['Date'].apply(lambda x: int(time.mktime(x.timetuple())) / 100000)

test['Date'] = test['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
test['Date'] = test['Date'].apply(lambda x: int(time.mktime(x.timetuple())) / 100000)

train_y = train_x['Weekly_Sales']
train_y = train_y.to_frame()
train_y = train_y.dropna().reset_index(drop = True)
train_x.drop('Weekly_Sales', axis = 1, inplace = True)

In [91]:
#Optional cell if you want to remove NaNs. This will mainly remove the MarkDown rows
#One hot code for everything data
storeType = train_x['Type']
encoding = pd.get_dummies(storeType)

train_x['A'] = encoding['A']
train_x['B'] = encoding['B']
train_x['C'] = encoding['C']
train_x.drop('Type', axis = 1, inplace = True)
train_x = train_x.dropna().reset_index(drop = True)

In [92]:
train_x.to_sql('train_x', engine, if_exists = 'replace', index = False)
train_y.to_sql('train_y', engine, if_exists = 'replace', index = False)
test.to_sql('test', engine, if_exists='replace', index = False)
baseTrain_x.to_sql('baseTrain_x', engine, if_exists='replace', index = False)
baseTrain_y.to_sql('baseTrain_y', engine, if_exists='replace', index = False)